1. mirror mount positions go from 0 to 1 (or any arbitrary continuous range really)

2. ion responses go from 0 to 100 photons per measurement round (or any arbitrary , small,
integer range)

In [1]:
from abc import ABC, abstractmethod
import math
import scipy.stats as stats


class IonSystem(ABC):
    _max_ion_response = 100

    @abstractmethod
    def move_mirror_to_position(self, pos: float) -> None:
        """
        Move position to pos. 0-1

        """
        ...

    @abstractmethod
    def measure_ion_response(self) -> int:
        """
        Return the ion response 0 to _max_ion_response

        """
        ...

class GaussianIonSystem(IonSystem):
    def __init__(self, peak_location: float, variance: float):
        # define constants
        self.peak_location: float = peak_location
        self.variance: float = variance
        self.pos: float = 0.

    @property
    def sigma(self) -> float:
        return math.sqrt(self.variance)

    def move_mirror_to_position(self, pos: float) -> None:
        """
        Move position to pos. 0-1

        """
        self.pos = pos

    def measure_ion_response(self) -> int:
        """
        Return the ion response 0 to _max_ion_response

        """
        return round(stats.norm.pdf(self.pos, self.peak_location, self.sigma))

0.3989422804014327